### Playing with `claude-v1.3-100k`

In [6]:
import os
import anthropic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import openai
import time
import tempfile
import shutil
from pathlib import Path
import subprocess
from tqdm import tqdm

In [7]:
client = anthropic.Client(api_key=os.environ['ANTHROPIC_API_KEY'])

## ContextualQA 

In [8]:
class ContextualQA:
    def __init__(self, client, model="claude-v1.3-100k"):
        self.client = client
        self.model = model
        self.context = ""
        self.questions = []
        self.responses = []

    def load_text(self, text):
        self.context = text

    def ask_question(self, question):
        leading_prompt = "Consider the text document below:"
        trailing_prompt = "Now answer the following question, use LaTex to format your answer."
        prompt = f"{anthropic.HUMAN_PROMPT} {leading_prompt}\n\n{self.context}\n\n{trailing_prompt}\n\n{anthropic.HUMAN_PROMPT} {question} {anthropic.AI_PROMPT}"
        response = self.client.completion_stream(
            prompt=prompt,
            stop_sequences=[anthropic.HUMAN_PROMPT],
            max_tokens_to_sample=6000,
            model=self.model,
            stream=False,
        )
        responses = [data for data in response]
        self.questions.append(question)
        self.responses.append(responses)
        return responses

    def clear_context(self):
        self.context = ""
        self.questions = []
        self.responses = []

    def __getstate__(self):
        state = self.__dict__.copy()
        del state["client"]
        return state

    def __setstate__(self, state):
        self.__dict__.update(state)
        self.client = None


In [9]:
with open('qa_system.pkl', 'rb') as f:
    qa_system = pickle.load(f)
    qa_system.client = client  # Manually set the client after loading

In [10]:
# Initialize the QA system
qa_system = ContextualQA(client)

# Load a text
with open('./gpts_are_gpts/main.tex', 'r') as f:
    paper_1 = f.read()

In [11]:
# Create a QA system
qa_system.load_text(paper_1)

### List of questions

In [12]:
# Warpup questions
# responses = qa_system.ask_question("What is the main argument of the paper?")
# print(responses[0]['completion'])

# responses = qa_system.ask_question("Can you summerize the paper in a few sentences?")
# print(responses[0]['completion'])

# responses = qa_system.ask_question("Can you tell me how GPT will change the world?")
# print(responses[0]['completion'])

# responses = qa_system.ask_question("Can you tell me the benefits of GPT on society and economy?")
# print(responses[0]['completion'])

In [13]:
# generated by GPT-4
questions = [
    "How has GPT impacted various industries economically?",
    "What are the potential economic benefits of using GPT?",
    "What are the potential economic drawbacks or risks of using GPT?",
    "How has GPT affected job markets and employment?",
    "How has GPT influenced social dynamics and interactions?",
    "What are the potential social benefits of using GPT?",
    "What are the potential social drawbacks or risks of using GPT?",
    "How has GPT impacted education and learning from a social perspective?",
    "How has GPT affected communication and information dissemination in society?",
    "Are there any case studies or examples that illustrate the economic and social impact of GPT?",
    "How might the use of GPT evolve in the future and what could be its potential economic and social implications?",
    "What are some potential solutions to the economic and social challenges posed by GPT?",
    "How does GPT compare to other AI technologies in terms of economic and social impact?",
    "What are the key takeaways from the paper regarding the economic and social impact of GPT?",
    "What are the ethical considerations surrounding the economic and social impact of GPT?",
    "How is GPT regulated, if at all, to mitigate its economic and social impact?",
    "How have businesses adapted to the economic changes brought about by GPT?",
    "How have communities and societies adapted to the social changes brought about by GPT?",
    "What predictions does the paper make about the future economic and social impact of GPT?",
    "What role do policymakers and regulators play in managing the economic and social impact of GPT?"
]


In [14]:
# # Ask a questions in questions list
# for question in tqdm(questions):
#     responses = qa_system.ask_question(question)

In [15]:
# Sanity check
questions = [
    "What is the title of the paper?",
    "Give methe abstract of the paper.",
    "Summarize the paper in a few sentences.",
    "Does the paper talk about Tax?",
    "Does the paper mention white collar workers?",
    "Does the paper mention blue collar workers?",
]

In [16]:
# Ask a questions in questions list
for question in tqdm(questions):
    responses = qa_system.ask_question(question)

100%|██████████| 6/6 [01:31<00:00, 15.25s/it]


In [17]:
# Save
with open('qa_system.pkl', 'wb') as f:
    pickle.dump(qa_system, f)

In [18]:
def latex_escape(text):
    text = text.replace("_", r"\_")
    text = text.replace("$", r"\$")
    return text

def qa_to_pdf(question, answer, output_filename):
    latex_template = r"""
    \documentclass{{standalone}}
    \usepackage[utf8]{{inputenc}}
    \usepackage{{amsmath}}
    \usepackage{{amssymb}}
    \usepackage{{hyperref}}
    \usepackage{{varwidth}}
    \usepackage{{adjustbox}}
    \begin{{document}}
    \begin{{adjustbox}}{{margin=5mm}}
    \begin{{varwidth}}{{\linewidth}}
    \textbf{{Question:}} \\
    {question} \\
    \textbf{{Answer:}} \\
    {answer}
    \end{{varwidth}}
    \end{{adjustbox}}
    \end{{document}}
    """

    answer = answer[0]['completion']
    latex_content = latex_template.format(question=latex_escape(question), answer=latex_escape(answer))

    with tempfile.TemporaryDirectory() as temp_dir:
        tex_file = Path(temp_dir) / "qa.tex"
        pdf_file = Path(temp_dir) / "qa.pdf"

        with open(tex_file, "w") as f:
            f.write(latex_content)

        subprocess.run(
            [
                "pdflatex",
                "-interaction=nonstopmode",
                "-output-directory",
                temp_dir,
                tex_file,
            ],
            check=True,
        )

        shutil.copy(pdf_file, output_filename)


In [19]:
def save_qa_to_pdf(qa_system, output_dir):
    for i, (question, answer) in enumerate(zip(qa_system.questions, qa_system.responses), 1):
        output_filename = os.path.join(output_dir, f"sanity_{i}.pdf")
        try:
            qa_to_pdf(question, answer, output_filename)
        except Exception as e:
            print(f"Failed to generate PDF for question {i}: {e}")

save_qa_to_pdf(qa_system, "./2")

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(/var/folders/gs/ftfl94c117q3mq9rqwfnx1mc0000gn/T/tmpxz28slld/qa.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2022/10/10 v1.3b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2023/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2023/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2023/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2023/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2023/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2023/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2023/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2023/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/